In [10]:
from sklearn.metrics import accuracy_score, f1_score
import pickle
import pathlib
import os
from xgboost import XGBClassifier



In [3]:
PROJECT_PATH = os.getcwd()
PROJECT_PATH = os.path.join(PROJECT_PATH, "..")
MODELS_DIR = pathlib.Path(PROJECT_PATH) / "store" / "models"

DATASET_DIR = pathlib.Path(PROJECT_PATH) / "data"

In [4]:
def load_data(dataset_name: str, split_ratio: float):
    path = DATASET_DIR / dataset_name / f"dataset_{split_ratio}.pkl"

    with open(path, "rb") as f:
        return pickle.load(f)

# Gesture Phase

In [8]:
X_train, y_train, X_test, y_test = load_data("gesture_phase", 0.2)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1777, 32), (1777,), (988, 32), (988,))

In [9]:
clf = XGBClassifier()
clf.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [12]:
preds = clf.predict(X_test)
accuracy_score(y_test, preds), f1_score(y_test, preds, average='weighted')

(0.6153846153846154, 0.5984407447266358)

In [14]:
with open(r"C:\Users\eden.yavin\Projects\Tabular-Cloud-ML\store\dataset\gesture_phase_0.2.pkl", "rb") as f:
    print(pickle.load(f))

{'train': [array([[-1.97918823,  0.86382736,  0.04923358, ...,  0.12312257,
         0.05773017,  0.02598728],
       [-0.00387541, -0.06456852, -0.04749759, ...,  0.12326053,
         0.05779486,  0.02489583],
       [-0.5907037 ,  1.39943731, -0.31360519, ...,  0.12504772,
         0.05863285,  0.0252568 ],
       ...,
       [ 0.71289846, -0.45646496, -0.08628519, ...,  0.12312257,
         0.05773017,  0.02598728],
       [-0.20713548,  0.37007397, -0.01031041, ...,  0.12326053,
         0.05779486,  0.02489583],
       [ 1.14746661,  1.1056628 , -0.20582642, ...,  0.11910935,
         0.05799439,  0.02610622]]), array([0, 2, 0, ..., 1, 0, 4])], 'test': [array([[ 2.79640224,  1.40675162,  1.24335887, ...,  0.12312257,
         0.05773017,  0.02598728],
       [ 0.09461413, -0.00367356, -0.00689974, ...,  0.12490573,
         0.05856627,  0.02636365],
       [ 0.52061317, -0.09750888,  0.17455672, ...,  0.12490573,
         0.05856627,  0.02636365],
       ...,
       [ 0.10489071, 